In [5]:
import re
import csv
from typing import List, Generator

def read_csv_multi_char_delimiter(file_path: str, delimiter: str, chunk_size: int = 1000) -> Generator[List[List[str]], None, None]:
    with open(file_path, 'r') as f:
        # Escape special regex characters in the delimiter
        escaped_delimiter = re.escape(delimiter)
        
        # Read and split the header
        header = re.split(escaped_delimiter, f.readline().strip())
        print(f"Header: {header}")
        expected_column_count = len(header)
        print(f"Header has {expected_column_count} columns")
        n_malformed_rows = 0
        chunk = []
        working_line = ''
        for row_num, line in enumerate(f, start=2): # Start from 2 as row 1 is header
            working_line += line.strip('\n')
            row = re.split(escaped_delimiter, working_line)
            if row[-1][0:4] != 'http':
                #print(f"Row {row_num} does not end with a URL")
                n_malformed_rows += 1
                continue

            if row[-1][0:4] == 'http' and len(row) != expected_column_count:
                print(f"Row {row_num} has {len(row)} columns, expected {expected_column_count}")
                n_malformed_rows += 1
            
            chunk.append(row)
            working_line = ''
            if len(chunk) == chunk_size:
                yield chunk
                chunk = []
        print(f"Number of malformed rows: {n_malformed_rows}")
        if chunk:  # Don't forget the last chunk
            yield chunk

# Usage
file_path = 'nyt_corpus.csv'
delimiter = '|||||'  # Replace with your actual multi-character delimiter
chunk_size = 1000

new_file_path = 'nyt_corpus_cleaned.csv'
header = ['title', 'section_name', 'snippet', 'lead_paragraph', 'year', 'month', 'web_url']
try:
    with open(new_file_path, 'a') as file:
        file.write(delimiter.join(header) + '\n')
    for chunk_num, chunk in enumerate(read_csv_multi_char_delimiter(file_path, delimiter, chunk_size), start=1):
        if chunk_num % 2000 == 0:
            print(f"Processing chunk {chunk_num}")
        # check if the chunk has the correct number of columns before writing to the new file
        for row in chunk:
            if len(row) != 7:
                raise ValueError(f"Row has {len(row)} columns, expected 7")
        
        string_to_write = '\n'.join([delimiter.join(row) for row in chunk]) + '\n'
        with open(new_file_path, 'a') as file:
            file.write(string_to_write)

except ValueError as e:
    print(f"Error in CSV structure: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Header: ['title', 'section_name', 'snippet', 'lead_paragraph', 'year', 'month', 'web_url']
Header has 7 columns
Processing chunk 2000
Processing chunk 4000
Processing chunk 6000
Processing chunk 8000
Processing chunk 10000
Number of malformed rows: 116799


In [17]:
# check cleaned file has no malformed rows

from tqdm import tqdm
from collections import defaultdict

print(new_file_path)

counter = defaultdict(lambda: defaultdict(int))

with open(new_file_path, 'r') as f:
    # Escape special regex characters in the delimiter
    escaped_delimiter = re.escape(delimiter)
    
    # Read and split the header
    header = re.split(escaped_delimiter, f.readline().strip())
    print(f"Header: {header}")
    expected_column_count = len(header)
    print(f"Header has {expected_column_count} columns")
    n_malformed_rows = 0
    for row_num, line in tqdm(enumerate(f, start=2)): # Start from 2 as row 1 is header
        row = re.split(escaped_delimiter, line)
        # Count the number of occurrences of each value in the section_name, year, and month columns
        for column in ['section_name', 'year', 'month']:
            counter[column][row[header.index(column)]] += 1

        # check year is numeric
        if not row[header.index('year')].isnumeric():
            print(f"Row {row_num} year is not numeric")
            n_malformed_rows += 1
        # check for year = 1851
        if row[header.index('year')] == '1851':
            print(f"Row {row_num} year is 1851")
            n_malformed_rows += 1

        if row[-1][0:4] != 'http':
            print(f"Row {row_num} does not end with a URL")
            n_malformed_rows += 1
            
        if len(row) != expected_column_count:
            print(f"Row {row_num} has {len(row)} columns, expected {expected_column_count}")
            n_malformed_rows += 1
    print(f"Number of malformed rows: {n_malformed_rows}")

nyt_corpus_cleaned.csv
Header: ['title', 'section_name', 'snippet', 'lead_paragraph', 'year', 'month', 'web_url']
Header has 7 columns


310226it [00:00, 619471.59it/s]

Row 178254 year is 1851
Row 178255 year is 1851
Row 178256 year is 1851
Row 178257 year is 1851
Row 178258 year is 1851
Row 178259 year is 1851
Row 178260 year is 1851
Row 178261 year is 1851
Row 178262 year is 1851
Row 178263 year is 1851


955616it [00:01, 737826.40it/s]

Row 731517 year is 1851
Row 731518 year is 1851
Row 731519 year is 1851
Row 731520 year is 1851
Row 731521 year is 1851
Row 731522 year is 1851
Row 731523 year is 1851
Row 731524 year is 1851
Row 731525 year is 1851
Row 731526 year is 1851


...

Row 10097873 year is not numeric
Row 10117706 year is not numeric
Row 10150693 year is not numeric
Row 10189874 year is not numeric
Row 10192507 year is not numeric
Row 10205460 year is not numeric


10381359it [00:15, 471981.01it/s]

Row 10319355 year is not numeric


10891105it [00:16, 640705.36it/s]

Number of malformed rows: 79


In [10]:
print(counter.keys())

for key in counter.keys():
    print(f"Unique values in {key}: {len(counter[key])}")
    print(f"Top 10 most common values in {key}:")
    for value, count in sorted(counter[key].items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"{value}: {count}")

dict_keys(['section_name', 'year', 'month'])
Unique values in section_name: 91
Top 10 most common values in section_name:
Archives: 6997368
Business Day: 734873
New York: 541275
U.S.: 366344
Sports: 358345
World: 321081
Opinion: 313981
Arts: 267545
Style: 111235
Books: 104160
Unique values in year: 100
Top 10 most common values in year:
1936: 173408
1937: 169383
1941: 168840
1930: 168794
1935: 168559
1934: 168145
1931: 165195
2006: 163382
1951: 162577
1950: 161566
Unique values in month: 12
Top 10 most common values in month:
10: 957520
05: 947436
03: 937173
01: 931721
04: 922900
11: 918846
06: 900381
07: 890903
12: 882774
08: 878949


In [13]:
print(counter['year'].keys())
print(len(counter['year'].keys()))
all_years_in_dataset = set(counter['year'].keys())
print(all_years_in_dataset)

dict_keys(['1930', '1931', '1851', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '|1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '|2010', '2011', '2012', '|2012', '2013', '|2013', '2014', '|2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])
100
{'1980', '2002', '2006', '1953', '1948', '1961', '1932', '1981', '|2012', '1977', '2011', '1957', '1944', '1997', '1983', '2013', '2008', '2010', '2015', '1994', '1949', '1979', '

In [14]:
years1930_2023 = set(f'{year}' for year in range(1930, 2024))
print(years1930_2023)

{'1980', '2002', '2006', '1953', '1948', '1961', '1932', '1981', '1977', '2011', '1957', '1944', '1997', '1983', '2013', '2008', '2010', '2015', '1994', '1949', '1979', '2012', '1992', '1938', '1960', '1970', '1993', '1956', '1963', '1986', '2004', '1969', '2016', '1972', '1967', '1988', '1968', '2020', '1940', '1974', '1998', '1989', '2021', '1936', '2001', '2019', '1945', '1990', '1955', '1996', '1976', '1952', '2023', '1951', '2007', '1959', '2017', '1975', '1999', '1984', '1962', '1947', '1941', '2009', '1971', '1954', '1991', '1950', '1978', '1946', '1966', '2014', '1943', '1964', '2018', '1931', '1965', '1937', '1933', '2000', '2022', '1958', '1934', '2005', '1995', '2003', '1942', '1985', '1935', '1939', '1973', '1987', '1930', '1982'}


In [15]:
all_years_in_dataset - years1930_2023

{'1851', '|1977', '|2010', '|2012', '|2013', '|2014'}